In [1]:
from libopt.dataset import Pipeline
from libopt.hook import PostgresLocal
from libopt.logger import Logger, PostgresLogger
from libopt.model import LinearModel, DNNModel

import pandas as pd
import numpy as np

## Load data from source and import to postgres

In [2]:
def import_to_postgres():
    url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'    
    column_names = ['MPG', 'Cylinders', 'Displacement', 
                    'Horsepower', 'Weight', 'Acceleration', 
                    'Model_Year', 'Origin'
                    ]
    column_names = [c.lower() for c in column_names]

    df = pd.read_csv(url, names=column_names,
                    na_values='?', comment='\t',
                    sep=' ', skipinitialspace=True)
    df.to_sql('auto_mpg', 
              PostgresLocal(host='172.19.0.2').engine, 
              if_exists='replace', 
              index=False)


## Import to postgres docker

`localhost:5432` or `postgres:5432` to connect to local container

In [3]:
# Import to postgres
import_to_postgres()

## Pipeline for Preprocessing

In [4]:
p = Pipeline()
engine = PostgresLocal(host='172.19.0.2').engine
df = pd.read_sql_table('auto_mpg', engine)
df = p.preprocess(df)
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1


In [5]:
# Custom for data set mpg only
df['origin'] = df['origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})
df = pd.get_dummies(df, columns=['origin'], prefix='', prefix_sep='')
train_x, train_y, test_x, test_y = p.prepare_train_test(df, y_column='mpg')

train_x.head(10)

,cylinders,displacement,horsepower,weight,acceleration,model_year,Europe,Japan,USA
146,4,90.0,75.0,2125.0,14.5,74,0,0,1
282,4,140.0,88.0,2890.0,17.3,79,0,0,1
69,8,350.0,160.0,4456.0,13.5,72,0,0,1
378,4,105.0,63.0,2125.0,14.7,82,0,0,1
331,4,97.0,67.0,2145.0,18.0,80,0,1,0
257,6,232.0,90.0,3210.0,17.2,78,0,0,1
310,4,89.0,60.0,1968.0,18.8,80,0,1,0
328,4,146.0,67.0,3250.0,21.8,80,1,0,0
125,6,198.0,95.0,3102.0,16.5,74,0,0,1
108,4,97.0,88.0,2279.0,19.0,73,0,1,0


## Train Model - Linear Regression

In [ ]:
# Logger for training model
logger = PostgresLogger(table_name = "m_model_execution")

# Linear Regression
ln = LinearModel()
_ = ln.train(train_x, train_y) 
eval_result = ln.eval(test_x, test_y, verbose=1)

# Write trained metadata to buffer
logger.write(**ln.metadata)
ln.metadata

## Train Model - Simple DNN

In [ ]:
dnn = DNNModel()
_ = dnn.train(train_x, train_y, verbose=0)
eval_result = dnn.eval(test_x, test_y, verbose=1)

# Write trained metadata to buffer
logger.write(**dnn.metadata)
dnn.metadata

## Write metadata to Postgres

- Use Logger to print out to stdout
- Use PostgresLogger to publish data to postgress

In [ ]:
# Publish buffer to postgres/stdout
logger.engine = engine
logger.publish(if_exists='append')

## Query trained metadata
Select all trained model with evaluation score smaller than  2.5 and

In [ ]:
m_df = pd.read_sql_table(logger.table_name, logger.engine)
m_df["date"] = pd.to_datetime(m_df['evaluate_on'], unit='s')\
                .dt.date
m_df.head()

In [ ]:
import datetime

# 7 date before and evaluate score smaller than 10
mae = 10
date_before = datetime.date.today() - datetime.timedelta(days=7)
m_df.query("date > @date_before and evaluate_score <= @mae")
